In [24]:
import pandas as pd

In [269]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [270]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [271]:
###Drop Nan Values
df=df.dropna()
df_test=df_test.dropna()
df['title'].head(5)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [273]:
## Get the Independent Features

X=df.drop('label',axis=1)
X.head(5)

#X_test=df_test.drop('label',axis=1)
len(df_test)

4575

In [29]:
## Get the Dependent features
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [233]:
X.shape
#X_test.shape

(18285, 4)

In [31]:
y.shape

(18285,)

In [32]:
import tensorflow as tf

In [33]:
tf.__version__

'1.13.1'

In [34]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

### Onehot Representation

In [107]:
messages=X.copy()

In [74]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [75]:
messages.reset_index(inplace=True)

In [76]:
import nltk
import re
from nltk.corpus import stopwords

In [77]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\658646\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
word_count=0
word_l=[]
#print(messages['title'][53])
for i in range(0, len(messages)):
    #print(i)
    
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    # review = re.sub('[nan]','Missing',messages['title'][i])
    #print(review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #Getting Unique no. of values in the corpus
    for word in review:
        if word != ' ' and word not in word_l:
             word_l.append(word)
             #word_count = word_count + 1
    review = ' '.join(review)
    corpus.append(review)


In [79]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [80]:
print(len(word_l)) #Total Unique no. of words-->Taken as input of Vocabulary size in embedded layer
print(len(corpus)) #No. of rows
print(word_l)#List of unique words

13931
18285
['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet', 'flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart', 'truth', 'might', 'get', 'fire', 'civilian', 'kill', 'singl', 'us', 'airstrik', 'identifi', 'iranian', 'jail', 'fiction', 'unpublish', 'stori', 'stone', 'death', 'adulteri', 'jacki', 'mason', 'hollywood', 'would', 'love', 'trump', 'bomb', 'north', 'korea', 'lack', 'tran', 'bathroom', 'exclus', 'video', 'beno', 'hamon', 'win', 'french', 'socialist', 'parti', 'presidenti', 'nomin', 'new', 'york', 'time', 'back', 'channel', 'plan', 'ukrain', 'russia', 'courtesi', 'associ', 'obama', 'organ', 'action', 'partner', 'soro', 'link', 'indivis', 'disrupt', 'agenda', 'bbc', 'comedi', 'sketch', 'real', 'housew', 'isi', 'caus', 'outrag', 'russian', 'research', 'discov', 'secret', 'nazi', 'militari', 'base', 'treasur', 'hunter', 'arctic', 'photo', 'offici', 'ye', 'paid', 'govern', 'troll', 'social', 'media', 'blog', 'forum', 'websit', 

In [ ]:
### Vocabulary size
voc_size=13931

In [81]:
import nltk
nltk.download('punkt')
minlen=len(corpus[0])
maxlen=-1
for doc in corpus:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
    if(minlen>len(tokens)):
         minlen=len(tokens)
#Count no of words in  the sentences which has  minimum and maximum number of words in corpus
print("The maximum number of words in any document is : ",maxlen)
print("The minimum number of words in any document is : ",minlen)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\658646\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The maximum number of words in any document is :  47
The minimum number of words in any document is :  0


In [110]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2694, 4044, 13159, 6571, 1098, 7593, 8357, 9481, 2385, 3771],
 [12118, 7942, 5241, 5182, 2480, 4919, 12327],
 [5459, 1835, 5706, 11390],
 [11348, 2394, 11021, 5652, 1503, 5567],
 [3833, 2480, 1886, 7132, 8611, 5119, 2480, 10609, 9649, 5092],
 [2592,
  11073,
  7077,
  5979,
  10033,
  10110,
  5725,
  11815,
  7932,
  8655,
  13582,
  3894,
  13283,
  7988,
  12327],
 [9288, 4542, 8042, 13069, 8567, 6294, 426, 3777, 5837, 5716, 5050],
 [8914, 2012, 2404, 10770, 5859, 11606, 10110, 2853, 5837, 5716, 5050],
 [7811, 13042, 13917, 4174, 12416, 8869, 13805, 5621, 10110, 5333],
 [7978, 4813, 13409, 12526, 4216, 1620, 8406, 8787],
 [5368, 10380, 11014, 2312, 12002, 8114, 2780, 5191, 11461, 570, 3862],
 [5652, 1230, 1098, 8869, 10110, 5859],
 [53, 8672, 2104, 10853, 10508, 10476, 1024, 5585, 5235],
 [8170, 3763, 7794, 2093, 7404, 7244, 13654, 5837, 5716, 5050],
 [1129, 8849, 10301, 7368, 8028, 5837, 5716, 5050],
 [4666, 11804, 4379, 643, 8328, 12461, 11584, 4003, 1000, 9212],
 [244, 1818, 79

### Embedding Representation

In [198]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(len(embedded_docs))

18285


In [180]:
embedded_docs[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0, 12118,  7942,  5241,  5182,  2480,
        4919, 12327])

In [131]:
## Creating Embedding layer model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
print(print(model.summary()))
embeddings=model.predict(embedded_docs)
print("Shape of embeddings : ",embeddings.shape)
print(embeddings[0])
#No.of param=input_size*output_size


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 20, 40)            557240    
Total params: 557,240
Trainable params: 557,240
Non-trainable params: 0
_________________________________________________________________
None
None
Shape of embeddings :  (18285, 20, 40)
[[ 0.04397923 -0.01953501  0.02800601  0.00971712 -0.04390607  0.04581448
   0.04511328 -0.02300984  0.02638203  0.03794909 -0.03914614  0.04300603
  -0.01734918  0.01131878  0.01959099  0.04962463  0.03154794 -0.02152759
  -0.0222912   0.01503238  0.04966955  0.0188261  -0.03657217 -0.00509945
  -0.01657516 -0.00759126  0.04841137  0.04518634 -0.01645695  0.00351161
   0.03310168 -0.01700792  0.00011664  0.02582991  0.04823038  0.00865283
  -0.01731538  0.04690911 -0.0346873   0.04808147]
 [ 0.04397923 -0.01953501  0.02800601  0.00971712 -0.04390607  0.04581448
   0.04511328 -0.02300984  0.02638203  0.03794

In [201]:
embedding_vector_features=32
e_model=Sequential()
e_model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
e_model.add(LSTM(128))
#No.of param=4((40*128)+(128*128)+128)
e_model.add(Dense(1,activation='sigmoid'))
e_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#print(model.summary())

In [154]:
len(embedded_docs),y.shape

(18285, (18285,))

In [155]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [156]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [170]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/5
12250/12250 [==============================] - 8s 655us/sample - loss: 0.3415 - acc: 0.8346 - val_loss: 0.2013 - val_acc: 0.9191
Epoch 2/5
12250/12250 [==============================] - 7s 549us/sample - loss: 0.1306 - acc: 0.9491 - val_loss: 0.1832 - val_acc: 0.9238
Epoch 3/5
12250/12250 [==============================] - 7s 546us/sample - loss: 0.0870 - acc: 0.9700 - val_loss: 0.2101 - val_acc: 0.9238
Epoch 4/5
12250/12250 [==============================] - 7s 564us/sample - loss: 0.0570 - acc: 0.9785 - val_loss: 0.2340 - val_acc: 0.9175
Epoch 5/5
12250/12250 [==============================] - 7s 555us/sample - loss: 0.0395 - acc: 0.9878 - val_loss: 0.3017 - val_acc: 0.9205


### Adding Dropout 

In [165]:
#Since Dropout reduce the accuracy not using the model with dropout
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
d_model=Sequential()
d_model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
d_model.add(Dropout(0.3))
d_model.add(LSTM(100))
d_model.add(Dropout(0.3))
d_model.add(Dense(1,activation='sigmoid'))
d_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
d_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

### Performance Metrics And Accuracy

In [186]:
y_pred=model.predict_classes(X_test)
print(len(X_test))

6035


In [172]:
from sklearn.metrics import confusion_matrix

In [173]:
confusion_matrix(y_test,y_pred)

array([[3102,  317],
       [ 163, 2453]], dtype=int64)

In [174]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9204639602319801

In [ ]:
#TESTING Data with new set of Documents (test.csv)

In [278]:
#Default Function to convert any documents of same type into Vectors
def DocToVector(X):
    messages=X.copy()
    messages.reset_index(inplace=True)
    ps = PorterStemmer()
    corpus_data = []
#word_count=0
    word_l=[]
#print(messages['title'][53])
    for i in range(0, len(messages)):
        #print(i)
    
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        # review = re.sub('[nan]','Missing',messages['title'][i])
        #print(review)
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #Getting Unique no. of values in the corpus
        for word in review:
            if word != ' ' and word not in word_l:
                 word_l.append(word)
             #word_count = word_count + 1
        review = ' '.join(review)
        corpus_data.append(review)
    
    voc_size_test=len(word_l)
    onehot_repr=[one_hot(words,voc_size_test)for words in corpus_data] 
    sent_length=20
    embedded_docs_test=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
    print(len(embedded_docs_test))
    return embedded_docs_test

In [279]:
#Convert test data into Embedded Vectors
Test_data=DocToVector(df_test)
#Output 4575x20xvoc_size Vector

4575


In [280]:
print(Test_data[1])
len(Test_data)

4575

In [281]:
#Mapping the title with corresponding output
for i in  range(len(Test_data)):
    y_pred=model.predict_classes(Test_data[[i]])  
    print(df_test.iloc[i:i+1,1:2]," ",y_pred)

                                               title
0  Specter of Trump Loosens Tongues, if Not Purse...   [[1]]
                                               title
2  #NoDAPL: Native American Leaders Vow to Stay A...   [[0]]
                                               title
3  Tim Tebow Will Attempt Another Comeback, This ...   [[1]]
                             title
4  Keiser Report: Meme Wars (E995)   [[1]]
                                               title
6  Pelosi Calls for FBI Investigation to Find Out...   [[1]]
                                     title
7  Weekly Featured Profile – Randy Shannon   [[1]]
                                                title
10  184 U.S. generals and admirals endorse Trump f...   [[1]]
                                   title
11  “Working Class Hero” by John Brennon   [[1]]
                                                title
12  The Rise of Mandatory Vaccinations Means the E...   [[1]]
                                  title
13  Commun

                                         title
88  Runway vs. Red Carpet - The New York Times   [[1]]
                                                title
89  D.C. Bar Will Offer Free Drinks Every Time Tru...   [[1]]
                                                title
90  Google Violence at Temple U Popular Across the...   [[0]]
                                                title
91  Wife of Orlando Gunman Is Charged Under Antite...   [[0]]
                                                title
92  A Sour Surprise for Public Pensions: Two Sets ...   [[0]]
                                                title
93  How to Save the Conversation at Your Post-Elec...   [[1]]
                                        title
94  The 5 Biggest Threats To Humanity Exposed   [[1]]
                                                title
95  Trump Will Issue New Travel Order Instead of F...   [[1]]
                                                title
96  Egyptian Court Clears Way for Hosni Mubarak’

                                                 title
206  Mom Fights Back After Son Gets Rejected From A...   [[1]]
                              title
207  Beyond the Anti-Trump Protests   [[1]]
                                                 title
208  Gorka Smites Haters, Radical Islamists in Jeru...   [[0]]
                                                 title
209  Teacher Accused of Having Sex with Underage St...   [[1]]
                                                 title
210  Trump Joins Line of Critics of Costly Fighter ...   [[1]]
                                                 title
211  Bernie Promises To Keep Fighting, Hold Clinton...   [[0]]
                                                 title
213  After Outcry, Murder Charge Filed in Death of ...   [[1]]
                                                 title
214  Living in a 5G World: Wireless Pollution is Ge...   [[1]]
                                           title
215  Polls Show Trump Winning Texas By Landsl

308  ’The Next Revolution? Steve Hilton is the Orig...   [[1]]
                                                 title
309  The Mysterious Metamorphosis of Chuck Close - ...   [[1]]
                                                 title
310  Spring Weather? Snowstorm? New York Schools to...   [[0]]
                                                 title
311  Death to the Fascist Insect! The SLA and the Cops   [[0]]
                                                 title
312  Ivana Says Young Donald Trump Was A Cry-Baby –...   [[1]]
                                                 title
313  Serena Williams and Novak Djokovic Sit Atop U....   [[1]]
                                                 title
314  White House: FBI Leaks ‘One of Many’ Reasons W...   [[1]]
                                                 title
316  After Senate Filibuster’s Death, Somber Lawmak...   [[1]]
                                                 title
317  A Tea Party Congressman Just Called For Armed ...  

416  Death Toll in Mistaken Bombing of Camp in Nige...   [[1]]
                                                 title
417  Another Day at a Monument to Democracy - The N...   [[1]]
                                                 title
418  Agents Accuse Feds of Covering Up Record Borde...   [[1]]
                                                 title
419  Oliver Stone: Trump-Russia Connection a ’Path ...   [[1]]
                                                 title
420  Trump’s Economic Vows May Be Harder to Hit Aft...   [[1]]
                                                 title
421  Russian Hackers Gained ‘Limited’ Access to R.N...   [[1]]
                                                 title
423  Justin Trudeau Criticized for Praising Fidel C...   [[1]]
                                                 title
425  A Sober Look at Everyday Life in Besieged Donetsk   [[1]]
                                                 title
426  Gennifer Flowers at Debate? Just Making a Poin...  

...

4679  YouTube Disables MILO Livestream Hours Before ...   [[0]]
                                                  title
4680  ¿Quién es Mariano Rajoy, el nuevo presidente d...   [[1]]
                                                  title
4681  Turkey Tumbling—-Crackdown Sweeps Through Busi...   [[1]]
                                                  title
4682  WOW! WHITE Liberals Suggest Blacks Are Too Stu...   [[1]]
                                                  title
4684  Trump’s Immigration Order Jolts Iraqis, U.S.’s...   [[1]]
                                           title
4687  US Uranium Weapons Have Been Used in Syria   [[1]]
                                                  title
4688  Paula Fox, Novelist Who Chronicled Dislocation...   [[1]]
                                                  title
4689  Ronald Lauder, Advocate of Art Restitution, Sa...   [[1]]
                                                  title
4690  Death Penalty for Dylann Roof in South Carolin..

                                                  title
4797  Published – ‘Dictionary’ – the post-truth edition   [[1]]
                  title
4798  Trump or Clinton?   [[1]]
                                                  title
4799  Pics: French Voters in London Turn Out to Cast...   [[0]]
                                                  title
4800  Reuters Donald Trump’s Hollywood Walk of Fame ...   [[1]]
                                                  title
4802  A Rio Table Tennis Lament: That’s the Way the ...   [[0]]
                                                  title
4803  CDC Blocks Testimony of Vaccine Whistleblower ...   [[1]]
                                                  title
4804  2016 Closes With 140 Cops Being Killed in the ...   [[0]]
                                                  title
4805  Trump Institute Offered Get-Rich Schemes With ...   [[0]]
                                                  title
4806  EXCLUSIVE – Islamic State Supporters Follo

                                                  title
4900  Schlagstöcke und Pyrotechnik sichergestellt: P...   [[0]]
                                                  title
4901  Paint and Switch? Did Alec Baldwin Pay $190,00...   [[0]]
                                                  title
4902  If You Have One Of These Old VHS Tapes It May ...   [[0]]
                                                  title
4903  Why Clam-Chowder Pizza Is the Best Kind - The ...   [[1]]
                                                  title
4904  Obama Says Colin Kaepernick Is ‘Exercising His...   [[1]]
                                                  title
4905  Sheldon Silver, Ex-New York Assembly Speaker, ...   [[0]]
                                                  title
4907  James Clapper: ’Warning Light’ Was on Regardin...   [[1]]
                                        title
4908  Meet The Man Who Started The Illuminati   [[1]]
                                                  title
4909

                                                  title
5004  Trump Weighs Infrastructure Bill but Keeps New...   [[0]]
                                                  title
5005  Breitbart News’ John Carney vs CNN’s Brian Ste...   [[0]]
                                                  title
5006  UK Jobs Market Can Be "Self-sufficient" After ...   [[1]]
                                                  title
5007  Trump Elected President, Thanks to 4 in 5 Whit...   [[1]]
                                             title
5008  Education System Broken: Let’s Try ‘Ed-Exit’   [[1]]
                                                  title
5009  Charlie Sheen on Trump: ’We Look Forward to Im...   [[1]]
                                                  title
5010  The Birth Bump – Do You Feel Like Were Born In...   [[1]]
                                                  title
5011  Comey Accused of Interfering in US Presidentia...   [[1]]
                                    title
5012  Ke

                                                  title
5108  Israel Refuses Participation In Paris Peace Co...   [[0]]
                                       title
5109  Опрос: Клинтон опережает Трампа на 14%   [[1]]
                                                  title
5110  Bill Cosby’s Books Join ‘Most Challenged’ List...   [[0]]
                                                  title
5111  ’Strategic Patience’ Is Over: Tillerson Floats...   [[1]]
                                                  title
5112  Israeli Supporters Of Donald Trump Rally At ‘J...   [[0]]
                                                  title
5113  Can Evolution Have a ‘Higher Purpose’? - The N...   [[1]]
                                                  title
5114  ’Three Explosions’ Near Borussia Dortmund Team...   [[1]]
                                                  title
5115  Hillary is So Unpopular, She Has to Pay Off Yo...   [[1]]
                                                  title
5116  